# MrVI Quick Start Tutorial
MrVI (Multi-resolution Variational Inference) is a package for analyzing multi-sample single-cell RNA-seq data. This tutorial will guide you through the main features of MrVI.

In [ ]:
!pip install --quiet scvi-colab
from scvi_colab import install

install()

In [ ]:
import os
import tempfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotnine as p9
import scanpy as sc
import scvi
from scvi.external import MrVI
import seaborn as sns

In [ ]:
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)
save_dir = tempfile.TemporaryDirectory()

In [ ]:
adata_path = os.path.join(save_dir.name, "haniffa_tutorial_subset.h5ad")

adata = sc.read(
    adata_path,
    backup_url="FILL" # Currently you can find this at s3://largedonor/haniffa_tutorial_subset.h5ad
)
adata



In [ ]:
sample_key="Site" # Nuisance covariate
batch_key="patient_id" # Sample identifier
MrVI.setup_anndata(adata, sample_key=sample_key, batch_key=batch_key)

In [ ]:
model = MrVI(adata)
model.train()

In [ ]:
plt.plot(model.history["elbo_validation"])
plt.xlabel("Epoch")
plt.ylabel("Validation ELBO")
plt.show()

In [ ]:
u = model.get_latent_representation()
u_mde = scvi.model.utils.mde(u)
adata.obsm["u_mde"] = u_mde
sc.pl.embedding(adata, basis="u_mde")

In [ ]:
z = model.get_latent_representation(give_z=True)
z_mde = scvi.model.utils.mde(z)
adata.obsm["z_mde"] = z_mde
sc.pl.embedding(adata, basis="z_mde")

In [ ]:
dists = model.get_local_sample_distances(
    adata, keep_cell=False, groupby="initial_clustering", batch_size=32
)

In [ ]:
sample_cov_keys = ["Status"]  # Replace with your sample covariate of interest
de_res = model.differential_expression(
    sample_cov_keys=sample_cov_keys,
)

In [ ]:
da_res = model.differential_abundance(sample_cov_keys=sample_cov_keys)
A_log_probs = da_res.Status_log_probs.loc[{"Status": "Covid"}]
B_log_probs = da_res.Status_log_probs.loc[{"Status": "Healthy"}]
A_B_log_prob_ratio = A_log_probs - B_log_probs

